In [1]:
import pandas as pd
import numpy as np
import vectorbt as vbt
import yfinance
import datetime

In [3]:
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days = 2)

In [4]:
btc_price = vbt.YFData.download(
    ['BTC-USD','ETH-USD'],
    start = start_date,
    end = end_date,
    missing_index=['drop'],
    interval = '1m'
    ).get('Close')

In [5]:
def custom_indicator(close, rsi_window=14, entry=30, exit=70):
    rsi = vbt.RSI.run(close, window = rsi_window).rsi
    trend = np.where(rsi>exit, -1, 0)
    trend = np.where(rsi<entry, 1, trend)
    return trend

In [7]:
ind = vbt.IndicatorFactory(
        class_name= 'Combination',
        short_name='comb',
        input_names=['close'],
        param_names=['rsi_window','entry','exit'],
        output_names=['value']
        ).from_apply_func(
                custom_indicator,
                rsi_window=14,
                entry=30,
                exit=70
                )

In [10]:
res = ind.run(
        btc_price,
        rsi_window=np.arange(10,40, step=3, dtype=int),
        entry=np.arange(10,40, step=4, dtype=int),
        exit=np.arange(60,80, step=4, dtype=int),
        param_product = True
        )

In [11]:
entries = res.value == 1.0
exits = res.value == -1.0

In [12]:
pf = vbt.Portfolio.from_signals(btc_price,entries,exits)

In [13]:
returns = pf.total_return()

In [15]:
print(returns.max())
print(returns.idxmax())

0.023525474221192725
(19, 10, 76, 'ETH-USD')


##### Saving btc_price as csv file and then using it
btc_price.to_csv('data01.csv')

In [17]:
btc_price = pd.read_csv('data01.csv')

In [19]:
btc_price.set_index('Datetime',inplace=True)